In [13]:
import pathlib
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from tensorflow import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import image_classification

# 4. Neural Networks for Image Classification 
In the last exercises we learned about neural networks and how they can correctly classify complicated data. So far we have only looked at datasets with relatively few features. In the iris dataset we looked at data with 4 features (sepal_length, sepal_width, petal_length, petal_width). However, in machine learning we want to deal with much more complex data such as images. In this notebook we will use neural networks to **classify images of handwritten digits** (numbers from 0 to 9). These images come from the so-called MNIST dataset. The size of the images here is **28 x 28 pixels and it is in black and white**.

## 4.1 Load and Visualize the Dataset
Let's first load the data and look at some examples of the data.

In [ ]:
# load the data and separate it into a train and a test set
(x_train, y_train), (x_test, y_test) = image_classification.load_mnist()
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# plot some images from the test set
image_classification.visualize_mnist(images = x_test, labels = y_test)

### Q4.1:
1. How many features do the images have?
2. Can you think of some features that are more or less important than others?
3. We now have 10 possible classes (before we had only two). Can you think of a way how we can use linear classifiers to distinguish between 10 classes?

## 4.2 Building the Classification Model
First we will implement a simple neural network in Keras that we can use to classify the images. As an activation function, we here use the **Softmax function** that is often used for multi-class classification. It outputs a probability for every class with that the Network thinks, the image belongs to that class.

In [15]:
# this is our neural network
network = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation = "softmax")
    ]
)

# we here compile the model using categorical crossentropy as a loss function. The categorical crossentropy is low if the model predicts the correct class.
network.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
# we use this to save our results of training
accuracies = {}

Before we start the training, let's check how well the network does without any training. The test accuracy is the percentage of images in the test set that our model predicted correctly.

In [ ]:
# this evaluates the network by predicting the labels of the test set and comparing the predictions and comparing these to the true labels
score = network.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {100*score[1]:.4}%")

### Q4.2
1. How many parameters does the network have? 
2. The test accuracy is not so great. What test accuracy would you have expected and why?

## 4.3 Training the Classification Model
We now run the training loop for 10 epochs (in one epoch we use every image of the dataset once to train the network). Training can take a minute so be patient.

In [ ]:
# this is the name of the run as it will show up in the logs
run_name = "first_try"
# this trains the network
out = network.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_test, y_test),
callbacks=[keras.callbacks.TensorBoard(f"../logs/{run_name}", update_freq="batch")])
# here we safe the train progress to compare it later
accuracies[run_name] = {"train": out.history["accuracy"], "test": out.history["val_accuracy"]}
# lets look at the test accuracy after training
score = network.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {100*score[1]:.4}%")
# lets plot how our training went
image_classification.plot_accuracies(accuracies)

### Q4.3
1. Which one of the graphs (train or test) is more important to assess the performance of our classifier?
2. Do you think a linear classifier would have a better, worse or the same test accuracy? Why?

## 4.4 Visualizing the results of Training
Now let's look at some of the images and the predicted label.

In [ ]:
# this predicts and visualizes the labels of the test set
predictions = network.predict(x_test, verbose=0)
image_classification.visualize_mnist(x_test, labels=y_test, predictions=predictions)

This already looks quite good. But there are still many images that are classified wrong. Let's look at some of them.

In [ ]:
# we here look for images that are classified wrong and show some of them
wrong_prediction_idxs = np.where(predictions.argmax(axis=-1) != y_test.argmax(axis=-1))
image_classification.visualize_mnist(x_test[wrong_prediction_idxs[:6]],
labels=y_test[wrong_prediction_idxs], predictions=predictions[wrong_prediction_idxs])

## 4.5 Try to get a better Accuracy!
Now you can play around with the number of layers and neurons. Change the network structure and see what happens. You can also try to change the number of training epochs or batch size. Give your runs meaningful names so you can tell them apart in the plots.

In [ ]:
 # give your run a meaningful name!
run_name = "meaningful_name"

# this is our neural network -> play around with the number and size of layers, but don't forget to add activation functions.
network = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation = "relu"),
        keras.layers.Dense(10, activation = "softmax"),
    ]
)

# this compiles the network -> you can try to use the optimizer "sgd" instead
network.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# this trains the network -> you can play around with the number of epochs or the batch size
out = network.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_test, y_test),
callbacks=[keras.callbacks.TensorBoard(f"../logs/{run_name}")])

# this saves the results
accuracies[run_name] = {"train": out.history["accuracy"], "test": out.history["val_accuracy"]}

# lets look at the test accuracy after training
score = network.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {100*score[1]:.4}%")

# this plots the accuracies over epochs
image_classification.plot_accuracies(accuracies)

### Q4.5:
1. What happens if you increase the number of epochs?
2. How does the performance change if we add a hidden layer with many neurons (e.g., 1000)?
3. What does the network do when we add a hidden layer with only one neuron?